In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load in csv
k_df = pd.read_csv("Best Cities for Startups.csv" , index_col=False , encoding = 'utf-8')
k_df.head()

,position,change in position from 2020,city,country,total score,quatity score,quality score,business score,sign of change in position
0,1,0,San Francisco Bay,United States,328.966,29.14,296.02,3.80,NaN
1,2,0,New York,United States,110.777,11.43,95.55,3.80,NaN
2,3,3,Beijing,China,66.049,5.01,58.61,2.43,+
3,4,1,Los Angeles Area,United States,58.441,11.23,43.41,3.80,+
4,5,2,London,United Kingdom,56.913,15.77,37.44,3.70,-


In [3]:
# Drop features
k_df.drop(["position","change in position from 2020","sign of change in position","total score", "quality score"], axis = 1 , inplace = True)

In [4]:
k_df

,city,country,quatity score,business score
0,San Francisco Bay,United States,29.14,3.80
1,New York,United States,11.43,3.80
2,Beijing,China,5.01,2.43
3,Los Angeles Area,United States,11.23,3.80
4,London,United Kingdom,15.77,3.70
...,...,...,...,...
995,Ouagadougou,Burkina Faso,0.02,0.02
996,Baghdad,Iraq,0.01,0.03
997,Mbabane,Swaziland,0.01,0.03
998,Conakry,Guinea,0.01,0.02


In [5]:
# Sort for only USA and change column for city to City for later join
usa_k_df = k_df[k_df["country"].str.contains("United States")!=False]
clean_usa = usa_k_df.rename(columns={"city":"City","quatity score":"quantity score"})
# Remove Text like Area, Bay, and some hypenated cities
clean_usa['City'] = clean_usa['City'].str.replace(r'Area|Bay|-Fort Worth|Durham|-Hoover|DC', "", regex=True)
clean_usa['City'] = clean_usa['City'].str.replace(r'Boise', "Boise City", regex=True)

In [6]:
# Drop Country feature
prep_k_df = clean_usa.drop('country',axis=1)
# prep_k_df = clean_usa.drop('country',axis=1).set_index("City",drop=True)

In [7]:
prep_k_df.head()

,City,quantity score,business score
0,San Francisco,29.14,3.8
1,New York,11.43,3.8
3,Los Angeles,11.23,3.8
5,Boston,5.50,3.8
11,Seattle,3.49,3.8


In [8]:
# Read in COL Index
c_df = pd.read_csv("advisorsmith_cost_of_living_index.csv", encoding = 'utf-8')

In [9]:
c_df.head()

,City,State,Cost of Living Index
0,Abilene,TX,89.1
1,Adrian,MI,90.5
2,Akron,OH,89.4
3,Alamogordo,NM,85.8
4,Albany,GA,87.3


In [10]:
# Drop the State feature
c_df.drop("State", axis = 1, inplace = True)

In [11]:
c_df

,City,Cost of Living Index
0,Abilene,89.1
1,Adrian,90.5
2,Akron,89.4
3,Alamogordo,85.8
4,Albany,87.3
...,...,...
505,Wheeling,84.1
506,New London,105.9
507,Daphne,96.6
508,Victoria,89.5


In [12]:
# Find Duplicates
c_df_bool = c_df.duplicated(subset = "City")
# Non Duplcates df
fc_df = c_df[~c_df_bool]

In [13]:
# Duplicates df
dupe_c_df = c_df[c_df_bool]
# Find the AVG and combine into one
mdcf = dupe_c_df.groupby("City").mean()
# Format COL Index Column
mdcf['Cost of Living Index'] = mdcf['Cost of Living Index'].map('{:,.1f}'.format)
# Reset Index
mdcf.reset_index(inplace=True)
# Append to non duplcates DF
ndf = fc_df.append(mdcf).reset_index(drop=True)

In [14]:
# strip the columns needed for merging
for df in (ndf, prep_k_df):
    # Strip the column(s) you're planning to join with
    df['City'] = df['City'].str.strip()

In [15]:
# Merge DFs
results = pd.merge(prep_k_df,ndf,how="left",on="City")

In [16]:
filtered_results = results.dropna(how="any")

In [17]:
filtered_results

,City,quantity score,business score,Cost of Living Index
0,San Francisco,29.14,3.80,178.6
1,New York,11.43,3.80,128.0
2,Los Angeles,11.23,3.80,140.6
3,Boston,5.50,3.80,132.6
4,Seattle,3.49,3.80,124.6
...,...,...,...,...
282,Winchester,0.01,0.12,97.2
283,Bowling Green,0.01,0.12,87.0
286,Lynchburg,0.01,0.10,93.6
287,Gulfport,0.01,0.10,86.8


In [18]:
# Find Duplicates and save for later review
cleaned_results = filtered_results.duplicated(subset = "City")
# Duplicates df
dupe_results = filtered_results[cleaned_results]

In [19]:
dupe_results

,City,quantity score,business score,Cost of Living Index
20,Portland,1.38,3.80,116.5
29,Columbus,0.76,3.80,91.5
39,Columbia,0.23,3.80,91.9
45,Burlington,0.26,3.80,112.8
52,Cleveland,0.51,3.18,90.0
57,Jacksonville,0.30,2.30,91.7
61,Richmond,0.33,1.95,95.0
64,Charleston,0.28,1.62,101.7
69,Lafayette,0.18,1.46,88.7
75,Rochester,0.16,1.31,96.6


In [20]:
# Non Duplcates df
fclean_results = filtered_results[~cleaned_results]

In [21]:
fclean_results

,City,quantity score,business score,Cost of Living Index
0,San Francisco,29.14,3.80,178.6
1,New York,11.43,3.80,128.0
2,Los Angeles,11.23,3.80,140.6
3,Boston,5.50,3.80,132.6
4,Seattle,3.49,3.80,124.6
...,...,...,...,...
282,Winchester,0.01,0.12,97.2
283,Bowling Green,0.01,0.12,87.0
286,Lynchburg,0.01,0.10,93.6
287,Gulfport,0.01,0.10,86.8


In [22]:
# Save to csv
fclean_results.to_csv("cleaned_data.csv")

In [23]:
# Find null 
r_bool = results["Cost of Living Index"].isnull()
nan_df = results[r_bool]

In [24]:
nan_df

,City,quantity score,business score,Cost of Living Index
27,Santa Barbara,0.25,3.80,NaN
65,Princeton,0.26,1.61,NaN
103,Franklin,0.09,0.67,NaN
111,Lewes,0.07,0.57,NaN
113,Annapolis,0.10,0.55,NaN
...,...,...,...,...
281,Conroe,0.01,0.12,NaN
284,Hooksett,0.01,0.12,NaN
285,Ashland,0.01,0.11,NaN
288,Hartland,0.01,0.10,NaN
